In [2]:
# pip install mysql-connector-python
!pip install --user -U scikit-learn==1.2.2
import mysql.connector
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import xgboost as xgb
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import joblib


In [3]:
con = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "G5thesis",
    database = "dataset_NF"
)
cursor = con.cursor()

In [4]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]
colnames

['IN_BYTES',
 'IN_PKTS',
 'OUT_BYTES',
 'OUT_PKTS',
 'TCP_FLAGS',
 'CLIENT_TCP_FLAGS',
 'SERVER_TCP_FLAGS',
 'FLOW_DURATION_MILLISECONDS',
 'DURATION_IN',
 'DURATION_OUT',
 'MIN_TTL',
 'MAX_TTL',
 'LONGEST_FLOW_PKT',
 'SHORTEST_FLOW_PKT',
 'MIN_IP_PKT_LEN',
 'MAX_IP_PKT_LEN',
 'SRC_TO_DST_SECOND_BYTES',
 'DST_TO_SRC_SECOND_BYTES',
 'SRC_TO_DST_AVG_THROUGHPUT',
 'DST_TO_SRC_AVG_THROUGHPUT',
 'NUM_PKTS_UP_TO_128_BYTES',
 'NUM_PKTS_128_TO_256_BYTES',
 'NUM_PKTS_256_TO_512_BYTES',
 'NUM_PKTS_512_TO_1024_BYTES',
 'NUM_PKTS_1024_TO_1514_BYTES',
 'TCP_WIN_MAX_IN',
 'TCP_WIN_MAX_OUT',
 'DNS_TTL_ANSWER',
 'Attack',
 'L4_DST_PORT_21',
 'L4_DST_PORT_22',
 'L4_DST_PORT_67',
 'L4_DST_PORT_80',
 'L4_DST_PORT_443',
 'L4_DST_PORT_500',
 'L4_DST_PORT_3306',
 'L4_DST_PORT_8080',
 'L4_DST_PORT_31808',
 'L4_DST_PORT_OTHERS',
 'L7_PROTO_7',
 'L7_PROTO_79',
 'L7_PROTO_92',
 'L7_PROTO_131',
 'L7_PROTO_178',
 'L7_PROTO_OTHERS']

In [5]:
labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() """,(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [6]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,73,1,113,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,2214,19,7969,32,223,218,31,4294936,16,31,...,0,0,0,0,0,0,0,0,0,1
2,6143,35,6844,35,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2629,20,9098,26,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,564,7,5114,9,222,222,90,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [7]:
base_models = []
df_0 = df.copy()
df_0['Attack'][df_0['Attack'] != 0] = 1
X_0 = df_0.drop(columns=['Attack'])
Y_0 = df_0[['Attack']]
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_0, Y_0, test_size=0.2, random_state=30)
base_model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
base_model.fit(X_train_0 , y_train_0 )
base_models.append(('model_' + '0', base_model))

In [8]:
df_copy = df.copy()

In [9]:
for i in range(1, 15):
    df_copy.truncate(before=-1, after=-1)
    df_copy = df.copy()
    df_copy['Attack'][df_copy['Attack'] != i] = 0
    df_copy.replace({'Attack': i}, {'Attack': 1}, inplace=True)
    X_copy = df_copy.drop(columns=['Attack'])
    Y_copy = df_copy[['Attack']]
    X_train_copy, X_test_copy, y_train_copy, y_test_copy = train_test_split(X_copy, Y_copy, test_size=0.2, random_state=30)
    base_model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
    base_model.fit(X_train_copy , y_train_copy )
    y_pred_copy = base_model.predict(X_test_copy)
    base_models.append(('model_' + str(i), base_model))
    results_copy = classification_report(y_test_copy,y_pred_copy,output_dict = True)
    df_res = pd.DataFrame(results_copy).transpose()
    print(i)
    print(df_res)
    print("\n")

1
              precision    recall  f1-score        support
0              1.000000  0.999996  0.999998  446738.000000
1              0.984127  1.000000  0.992000     124.000000
accuracy       0.999996  0.999996  0.999996       0.999996
macro avg      0.992063  0.999998  0.995999  446862.000000
weighted avg   0.999996  0.999996  0.999996  446862.000000


2
              precision    recall  f1-score        support
0              0.999998  0.956970  0.978011  446553.000000
1              0.015776  0.996764  0.031061     309.000000
accuracy       0.956998  0.956998  0.956998       0.956998
macro avg      0.507887  0.976867  0.504536  446862.000000
weighted avg   0.999317  0.956998  0.977356  446862.000000


3
              precision    recall  f1-score        support
0              0.999998  0.956651  0.977844  446697.000000
1              0.008398  0.993939  0.016656     165.000000
accuracy       0.956664  0.956664  0.956664       0.956664
macro avg      0.504198  0.975295  0.497250  4

In [10]:
X = df.drop(columns=['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=30)

In [11]:
from xgboost import XGBClassifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=DecisionTreeClassifier(),cv="prefit")
stacking_model.fit(X_train, y_train)
y_pred = stacking_model.predict(X_test)

In [12]:
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()

In [13]:
df

,precision,recall,f1-score,support
0,0.987582,0.993109,0.990337,343770.000000
1,1.000000,0.983871,0.991870,124.000000
2,0.047325,0.074434,0.057862,309.000000
3,0.075581,0.078788,0.077151,165.000000
4,0.979000,0.971405,0.975188,19724.000000
5,1.000000,0.984962,0.992424,399.000000
6,0.998250,1.000000,0.999124,3423.000000
7,1.000000,1.000000,1.000000,5491.000000
8,1.000000,1.000000,1.000000,64898.000000
9,1.000000,1.000000,1.000000,10.000000


In [28]:
# from imblearn.ensemble import BalancedRandomForestClassifier

# model_dos_hulk = BalancedRandomForestClassifier(n_estimators = 30, random_state = 30)
# # Train the model
# model_dos_hulk.fit(X_train,y_train)
# y_pred = model_dos_hulk.predict(X_test)
# results = classification_report(y_test,y_pred,output_dict = True)
# df = pd.DataFrame(results).transpose()
